# Additional Modelling Challenges

## Job-Shop Scheduling

We need to schedule the processing of the tasks from different jobs in several machines.

Each task is labelled by a pair of numbers (`m`, `p`) where `m` is the number of the machine and `p` is the processing time of the task.

There are three jobs: For example, job `0` has three tasks. The first, `(0, 3)`, must be processed on machine `0` in `3` units of time.

```
job 0 = [(0, 3), (1, 2), (2, 2)]
job 1 = [(0, 2), (2, 1), (1, 4)]
job 2 = [(1, 4), (2, 3)]
```

There are two types of constraints for the job-shop problem:

1. Precedence constraint: for any two consecutive tasks in the same job, the first must be completed before the second can be started.
2. No-overlap constraint: a machine can't work on two tasks at the same time.

The objective of the job-shop problem is to minimize the makespan: the length of time from the earliest start time of the jobs to the latest end time.

In [1]:
import cpmpy as cp

# Parameters
# (machine, processing_time)
tasks_by_job = [[(0, 3), (1, 2), (2, 2)],
                [(0, 2), (2, 1), (1, 4)],
                [(1, 4), (2, 3)]]

n_jobs = len(tasks_by_job)
n_machines = max(task[0] for job in tasks_by_job for task in job) + 1
max_time = sum(task[1] for job in tasks_by_job for task in job)

model = cp.Model()

# Decision variables
# start[j][t] represents the start time of task t in job j
start = [[cp.intvar(0, max_time, name=f"start_{j}_{t}")
          for t in range(len(job))]
         for j, job in enumerate(tasks_by_job)]
makespan = cp.intvar(0, max_time, name="makespan")

# Constraints
# 1. Precedence constraint: for any two consecutive tasks in the same job, the first must be completed before the second can be started.
for j, job in enumerate(tasks_by_job):
    for t in range(len(job) - 1):
        model += start[j][t] + job[t][1] <= start[j][t + 1]

# 2. No-overlap constraint: a machine can't work on two tasks at the same time.
tasks_by_machine = [[] for _ in range(n_machines)]
for j, job in enumerate(tasks_by_job):
    for t, (machine, proc_time) in enumerate(job):
        tasks_by_machine[machine].append((j, t, proc_time))

for machine_tasks in tasks_by_machine:
    for i, (j1, t1, p1) in enumerate(machine_tasks):
        for j2, t2, p2 in machine_tasks[i + 1:]:
            model += ((start[j1][t1] + p1 <= start[j2][t2]) | 
                     (start[j2][t2] + p2 <= start[j1][t1]))

# The objective of the job-shop problem is to minimize the makespan: the length of time from the earliest start time of the jobs to the latest end time.
for j, job in enumerate(tasks_by_job):
    last_task_idx = len(job) - 1
    model += start[j][last_task_idx] + job[last_task_idx][1] <= makespan

model.minimize(makespan)

if model.solve():
    print("Solution found!")
    print(f"Makespan: {makespan.value()}")

    for j, job in enumerate(tasks_by_job):
        print(f"\nJob {j}:")
        for t, (machine, proc_time) in enumerate(job):
            print(f"  Task on machine {machine}: start={start[j][t].value()}, "
                  f"end={start[j][t].value() + proc_time}")
else:
    print("No solution found.")

Solution found!
Makespan: 11

Job 0:
  Task on machine 0: start=2, end=5
  Task on machine 1: start=5, end=7
  Task on machine 2: start=7, end=9

Job 1:
  Task on machine 0: start=0, end=2
  Task on machine 2: start=2, end=3
  Task on machine 1: start=7, end=11

Job 2:
  Task on machine 1: start=0, end=4
  Task on machine 2: start=4, end=7
